# Deep Neural Network for MNIST Classification

### Import the relavant packages

In [1]:
import numpy as np 
import tensorflow as tf 
import tensorflow_datasets as tfds

C:\Users\Manuel\AppData\Roaming\Python\Python312\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


### Data

In [10]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)


In [3]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255. 
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)
    
BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)

train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)

validation_data = validation_data.batch(num_validation_samples)

test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))





# Model


### Outline the model

In [4]:
input_size = 784
output_size = 10
hidden_layer = 200
 
model = tf.keras.Sequential([
                              tf.keras.layers.Flatten(input_shape=(28,28,1)),
                              tf.keras.layers.Dense(hidden_layer, activation='relu'),
                              tf.keras.layers.Dense(hidden_layer, activation='tanh'),
                              tf.keras.layers.Dense(hidden_layer, activation='sigmoid'),
                              tf.keras.layers.Dense(output_size, activation='softmax')
     
 ])

C:\Users\Manuel\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\reshaping\flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#### Choose the optimizer and lost function

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training

In [6]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 27s - 51ms/step - accuracy: 0.9104 - loss: 0.3163 - val_accuracy: 0.9593 - val_loss: 0.1353
Epoch 2/5
540/540 - 16s - 30ms/step - accuracy: 0.9672 - loss: 0.1063 - val_accuracy: 0.9768 - val_loss: 0.0788
Epoch 3/5
540/540 - 18s - 33ms/step - accuracy: 0.9788 - loss: 0.0696 - val_accuracy: 0.9795 - val_loss: 0.0661
Epoch 4/5
540/540 - 11s - 20ms/step - accuracy: 0.9833 - loss: 0.0521 - val_accuracy: 0.9832 - val_loss: 0.0540
Epoch 5/5
540/540 - 11s - 20ms/step - accuracy: 0.9878 - loss: 0.0386 - val_accuracy: 0.9853 - val_loss: 0.0432


# Test the model

In [7]:
test_loss, test_accuracy = model.evaluate(test_data)


print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'. format(test_loss, test_accuracy*100))

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.9780 - loss: 0.0740
Test loss: 0.07. Test accuracy: 97.80%
